# Setup

## Imports

In [15]:
from typing import \
    Callable, \
    List, \
    NewType, \
    Tuple


import numpy as np

### Temporary

In [33]:
from collections import Counter

## Types

In [12]:
Encounter = NewType('Encounter', int)
Position = NewType('Position', Tuple[int, int])
Reward = NewType('Reward', int)
Move = NewType('Move', Tuple[Position, Reward])
History = NewType('History', List[Move])

## Constants

In [16]:
INIT_POS = Position((0, 0))
NUM_MOVES = 10

# Encounter Types
EMPTY = 0
MONSTER = 1
TREASURE = 2

ENCOUNTER_TO_REWARD = {
    EMPTY: 0,
    MONSTER: -10,
    TREASURE: 10
}

# Classes

## Cell Class

In [89]:
class Cell:
    def __init__(self, p_empty: float, p_monster: float, p_treasure: float):
        """
        Init with probabilities set
        """
        self._p_empty = p_empty
        self._p_monster = p_monster
        self._p_treasure = p_treasure
        
    @property
    def p_empty(self):
        return self._p_empty
        
    @property
    def p_monster(self):
        return self._p_monster
        
    @property
    def p_treasure(self):
        return self._p_treasure
    
    def __call__(self) -> Reward:
        """
        Returns reward
        """
        x = np.random.rand()
        
        if x < self.p_empty:
            return ENCOUNTER_TO_REWARD[EMPTY]
        if x < self.p_monster + self.p_empty:
            return ENCOUNTER_TO_REWARD[MONSTER]
        return ENCOUNTER_TO_REWARD[TREASURE]
    
    def __str__(self) -> str:
        p_empty = self.p_empty
        p_monster = self.p_monster
        p_treasure = self.p_treasure
        
        return f'Probability of Empty: {p_empty:.2f}\nProbability of Monster: {p_monster:.2f}\nProbability of Treasure: {p_treasure:.2f}'

### Testing

In [100]:
cell = Cell(0.2, 0.7, 1)

Counter([cell() for _ in range(100000)])

Counter({-10: 70284, 10: 10023, 0: 19693})

## Grid Class

In [69]:
class Grid:
    def __init__(self, cells: List[List[Cell]]):
        """
        Init with cells
        """
        
        self.cells = cells
        self.num_rows = len(cells)
        self.num_cols = len(cells[0])
    
    def valid_next_positions(self, position: Position) -> List[Position]:
        """
        Returns list of valid next cells
        """
        
        i, j = position
        
        valid_is = [i]
        valid_js = [j]
        
        if i > 0:
            valid_is.append(i - 1)
        if i < self.num_rows - 1:
            valid_is.append(i + 1)
        if j > 0:
            valid_js.append(j - 1)
        if j < self.num_cols - 1:
            valid_js.append(j + 1)
            
        return [Position((i, j)) for i in valid_is for j in valid_js if Position((i, j)) != position]
        
    def __getitem__(self, position: Position):
        i, j = position
        return self.cells[i][j]

### Testing

In [92]:
I, J = 3, 3

cells = []
for _ in range(I):
    row = []
    for _ in range(J):
        x = np.random.rand()
        y = np.random.rand() * (1 - x)
        z = 1 - x - y
        
        probabilities = [x, y, z]
        np.random.shuffle(probabilities)
        
        row.append(Cell(*probabilities))
        
    cells.append(row)

grid = Grid(cells)

print(grid.valid_next_positions((0,0)))
print(grid[1, 1])
print(grid[1, 2] == cells[1][2])

[(0, 1), (1, 0), (1, 1)]
Probability of Empty: 0.18
Probability of Monster: 0.04
Probability of Treasure: 0.78
True


## ModelCell and ModelGrid Classes

In [94]:
class ModelCell(Cell):
    def __init__(self):
        """
        Init, tracking with num encounters etc
        """
        self.num_encounters = 0
        self.encounters = {
            Encounter(EMPTY): 0,
            Encounter(MONSTER): 0,
            Encounter(TREASURE): 0
        }
        
        super().__init__(0, 0, 0)
    
    @Cell.p_empty.getter
    def p_empty(self):
        if self.num_encounters == 0:
            return 1/3
        
        return self.encounters[Encounter(EMPTY)] / self.num_encounters

    @Cell.p_monster.getter
    def p_monster(self):
        if self.num_encounters == 0:
            return 1/3
        
        return self.encounters[Encounter(MONSTER)] / self.num_encounters
        
    @Cell.p_treasure.getter
    def p_treasure(self):
        if self.num_encounters == 0:
            return 1/3
        
        return self.encounters[Encounter(TREASURE)] / self.num_encounters
    
    def update(self, encounter: Encounter):
        """
        Update encounters tracking with encounter
        """
        
        self.num_encounters += 1
        self.encounters[encounter] += 1
        
class ModelGrid(Grid):
    def __init__(self, cells: List[List[ModelCell]]):
        super.__init__(cells)
    
    def update(self, position: Position, encounter: Encounter):
        """
        Update cell in position with encounter e.g. monster
        """
        
        self[position].update(encounter)

### Testing

In [111]:
cell = ModelCell()

print(Counter([cell() for _ in range(10000)]))

cell.update(EMPTY)
print(Counter([cell() for _ in range(10000)]))
cell.update(np.random.choice([MONSTER, TREASURE]))
print(Counter([cell() for _ in range(10000)]))

Counter({10: 3377, -10: 3333, 0: 3290})
Counter({0: 10000})
Counter({-10: 5063, 0: 4937})


## Game Class

In [7]:
class Game:
    def __init__(self, grid: Grid):
        """
        Init with grid
        """
        pass
    
    def print(self, pos: Position) -> str:
        """
        Pretty print board with player's position
        """
        pass

## Player Class

In [8]:
StrategyFunction = NewType('StrategyFunction', Callable[[Grid, Position], Cell])

class Player:
    def __init__(self, game: Game, strategy_fn: StrategyFunction, num_moves:int = NUM_MOVES):
        """
        Init with strategy function, grid model, history
        """
        pass
    
    def next_move(self):
        """
        Perform next move according to strategy function
        """
        pass
    
    def play(self) -> History:
        """
        Play until remaining moves expended, then return History
        """
        pass

# Strategy Functions

## $\varepsilon\text{-greedy}$

In [9]:
def epsilon_greedy_maker(epsilon: float) -> StrategyFunction:
    def epsilon_greedy(model: Grid, position: Position) -> Position:
        pass

## $\alpha\text{-index}$, Maximin, Maximax

In [11]:
def alpha_index_maker(alpha: float) -> StrategyFunction:
    def alpha_index(model: Grid, position: Position) -> Position:
        pass
    
maximin: StrategyFunction = alpha_index_maker(0)
maximax: StrategyFunction = alpha_index_maker(1)